In [41]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [42]:
application_df['ASK_AMT'].value_counts()

ASK_AMT
5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: count, Length: 8747, dtype: int64

In [43]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
# Additionally drop: 'STATUS', 'SPECIAL_CONSIDERATIONS'
to_drop = ['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS']
application_df.drop(to_drop, inplace=True, axis=1)
application_df.head(2)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1


In [44]:
# Replace all application types with count below 100
application_types_to_replace = ["T13", "T12", "T2", "T25", "T14", "T29", "T15", "T17", "T9"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [45]:
# Get all CLASSIFICATION values with count below 1000
filt_df = application_df.groupby("CLASSIFICATION").filter(lambda classification: len(classification) < 400)
classifications_to_replace = filt_df["CLASSIFICATION"].unique()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64

In [46]:
# Bin extravagently large income amounts together
application_df['INCOME_AMT'].value_counts()

INCOME_AMT
0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: count, dtype: int64

In [47]:
# Get all large INCOME_AMT
amts_to_replace = ['10M-50M', '5M-10M', '50M+']

# Replace in dataframe
for cls in amts_to_replace:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(cls,"5M+")

# Check to make sure binning was successful
application_df['INCOME_AMT'].value_counts()

INCOME_AMT
0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
5M+                564
10000-24999        543
Name: count, dtype: int64

In [48]:
# Convert categorical data to numeric with `pd.get_dummies`
with_dummies = pd.get_dummies(application_df)
with_dummies.head(2)

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_5M+
0,5000,1,False,True,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
1,108590,1,False,False,False,True,False,False,False,False,...,True,False,False,False,True,False,False,False,False,False


In [49]:
# Split our preprocessed data into our features and target arrays
X = with_dummies.drop(["IS_SUCCESSFUL"], axis=1).values
y = with_dummies["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [50]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features_num = len(X_train[0])
layer1_nodes = 60
layer2_nodes = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=layer1_nodes, input_dim=input_features_num, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=layer2_nodes, activation="softmax"))

# Output layer; sigmoid chosen due to binary nature of target
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 60)                2400      
                                                                 
 dense_16 (Dense)            (None, 30)                1830      
                                                                 
 dense_17 (Dense)            (None, 1)                 31        
                                                                 
Total params: 4261 (16.64 KB)
Trainable params: 4261 (16.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [51]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

In [52]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=150)

Epoch 1/150
804/804 [==============================] - 2s 2ms/step - loss: 0.6005 - accuracy: 0.7156
Epoch 2/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5699 - accuracy: 0.7313
Epoch 3/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5589 - accuracy: 0.7330
Epoch 4/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5536 - accuracy: 0.7341
Epoch 5/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5514 - accuracy: 0.7346
Epoch 6/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5495 - accuracy: 0.7352
Epoch 7/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5479 - accuracy: 0.7346
Epoch 8/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5465 - accuracy: 0.7355
Epoch 9/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5461 - accuracy: 0.7353
Epoch 10/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5452 - accura

In [53]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5625 - accuracy: 0.7213 - 437ms/epoch - 2ms/step
Loss: 0.5624985098838806, Accuracy: 0.7212827801704407


In [54]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
